In [1]:
import boto3
import sagemaker
import datetime
from time import gmtime, strftime

# Specify your AWS Region
aws_region='us-east-1'
account_id = 259504422128

enviroment = 'sbx'
repository_ecr = 'plata-model-registry'

model_prefix = 'model-plata'
model_name = f'{model_prefix}'

#Create a variable w/ the model S3 URI
s3_bucket = f'zenpli-model-artifacts-sbx' # Provide the name of your S3 bucket
bucket_prefix='artifacts'
model_id = f'model.tar.gz'
model_s3_key = f"{bucket_prefix}/{model_id}"

# Create a low-level SageMaker service client.

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

# Role to give SageMaker permission to access AWS services.
sagemaker_role= sagemaker.get_execution_role()

# Specify S3 bucket w/ model
model_url = f"s3://{s3_bucket}/{model_s3_key}"
print(model_url)

# Specify an AWS container image. 

container = f'{account_id}.dkr.ecr.{aws_region}.amazonaws.com/{repository_ecr}:{model_name}' 
print(container)

#container = '259504422128.dkr.ecr.us-east-1.amazonaws.com/plata-model-registry:model-plata'


# Create Model 

create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': model_url,
    })

# Create Endpoint Configuration 

endpoint_config_name = f"{model_name}-cfg"

endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name, 
            "InstanceType": "ml.t2.medium", # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

endpoint_name = f'{model_prefix}-endpoint'

create_endpoint_response = sagemaker_client.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name) 

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
s3://zenpli-model-artifacts-sbx/artifacts/model.tar.gz
259504422128.dkr.ecr.us-east-1.amazonaws.com/plata-model-registry:model-plata
Created EndpointConfig: arn:aws:sagemaker:us-east-1:259504422128:endpoint-config/model-plata-cfg
